<a href="https://colab.research.google.com/github/EndreGuljas/AER850_Project3/blob/main/AER850PROJECT3_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''Step 1: Object Masking'''
import cv2
import numpy as np
from matplotlib import pyplot as plt

path = "/content/drive/MyDrive/AER850_Project3/motherboard_image.JPEG"
img_real = cv2.imread(path, cv2.IMREAD_COLOR)
img_real = cv2.rotate(img_real, cv2.ROTATE_90_CLOCKWISE)

img = cv2.imread(path, cv2.IMREAD_COLOR)
img = cv2.GaussianBlur(img, (47, 47), 4)
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img_gray = cv2.adaptiveThreshold(img_gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 55, 7)
img_gray = cv2.rotate(img_gray, cv2.ROTATE_90_CLOCKWISE)

edges = cv2.Canny(img_gray, 50, 300)
edges = cv2.dilate(edges, None, iterations=10)

contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

mask = np.zeros_like(img_real)
cv2.drawContours(image=mask, contours=[max(contours, key=cv2.contourArea)], contourIdx=-1, color=(255, 255, 255), thickness=cv2.FILLED)

masked_img = cv2.bitwise_and(mask, img_real)

masked_img_rgb = cv2.cvtColor(masked_img, cv2.COLOR_BGR2RGB)

plt.imshow(masked_img_rgb)
plt.axis('off')
plt.show()


In [ ]:
!pip install ultralytics

In [ ]:
'''Step 2: YOLOv8 Training'''
from ultralytics import YOLO

model = YOLO('yolov8n.pt')
results = model.train(data='/content/drive/MyDrive/AER850_Project3/data/data.yaml', epochs=150, batch=2, imgsz=1024, name='pcb_identifier')

In [ ]:
'''Step 3: YOLOv8 Evaluation'''
from google.colab.patches import cv2_imshow

# Load your trained model
model_path = 'XXX'
model = YOLO(model_path)

# Define the path to evaluation images
image_paths = [
    '/content/drive/MyDrive/AER850_Project3/data/evaluation/ardmega.jpg',
    '/content/drive/MyDrive/AER850_Project3/data/evaluation/arduno.jpg',
    '/content/drive/MyDrive/AER850_Project3/data/evaluation/rasppi.jpg'
]

# Iterate over each image path, predict, and display results
for image_path in image_paths:
    # Make predictions with the model
    results = model.predict(image_path, imgsz=1024, conf=0.35)

    # Get the plotted image with detections
    result_image = results[0].plot()

    # Display the image
    print(f"\nResults for: {image_path.split('/')[-1]}")
    cv2_imshow(result_image)

    # Print detection statistics
    boxes = results[0].boxes
    print(f"Total components detected: {len(boxes)}")

    # Print detection details
    for box in boxes:
        class_id = int(box.cls[0])
        class_name = model.names[class_id]
        confidence = float(box.conf[0])
        print(f"Class: {class_name}, Confidence: {confidence:.2f}")

    print("-" * 50)  # Separator between images